In [2]:
import datetime as datetime
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt

#Permite ver los graficos sin colocar plt.show
%matplotlib inline 

plt.style.use('default')
#plt.rcParams['figure.figsize'] = (15, 5)


#Funcion para calcular distancias entre coordenadas (lat, lon)
from math import sin,cos,sqrt,asin,pi

#Formula del Harvesine sacada de
#https://joseguerreroa.wordpress.com/2012/11/13/distancia-entre-dos-puntos-de-la-superficie-terrestre-mediante-la-formula-de-haversine-con-python/
#coord = (lat, long)
def distancia(coord1, coord2):
    rTierra = 6374000 #metros
    c = pi/180 #para pasar a raddianes
    d = 2*rTierra*asin(sqrt(sin(c*(coord2[0]-coord1[0])/2)**2 + cos(c*coord1[0])*cos(c*coord2[0])*sin(c*(coord2[1]-coord1[1])/2)**2))
    return d

#reg = created_on, place_name, lat, lon, price_usd_per_m2
def distanciaANuevasEstaciones(reg):
    created_on, place_name, lat, lon, price_usd_per_m2 = reg
    geoPos = lat, lon
    #Ubicacion de las estaciones
    Cordoba = (-34.598691, -58.403841) #Inicio Obras: 17 ene 2012 - Inauguracion: 18 dic 2015
    SantaFe = (-34.594848, -58.402517) #Inicio Obras: 17 ene 2012 - Inauguracion: 12 jul 2016
    LasHeras = (-34.587879, -58.397970) #Inicio Obras: 17 ene 2012 - Inauguracion: 18 dic 2015

    distACordoba = distancia(geoPos, Cordoba)
    distASantaFe = distancia(geoPos, SantaFe)
    distALasHeras = distancia(geoPos, LasHeras)
    

    return min(distACordoba, distASantaFe, distALasHeras)

data17 = pd.read_csv("properati-AR-2017-08-01-properties-sell.csv", low_memory = False)
data16 = pd.read_csv("properati-AR-2016-12-01-properties-sell.csv", low_memory = False)
data15 = pd.read_csv("properati-AR-2015-12-01-properties-sell.csv", low_memory = False)
data14 = pd.read_csv("properati-AR-2015-12-01-properties-sell.csv", low_memory = False)
frames = [data14, data15, data16, data17]
data = pd.concat(frames)
data.head()

FileNotFoundError: File b'properati-AR-2016-12-01-properties-sell.csv' does not exist

In [ ]:
data.drop_duplicates(subset=['lat', 'lon', 'price_usd_per_m2'])
data.info()

In [ ]:
#Solo me quedo con CAP FED y GBA
dataCapGba = data.loc[(data.state_name.str.contains('Capital Federal') | data.state_name.str.contains('G.B.A') ), ['created_on','operation', 'property_type', 'state_name', 'place_name', 'lat', 'lon', 'price', 'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor', 'rooms', 'expenses', 'description']]

In [ ]:
dataCapGbaMenoresA10K = dataCapGba.loc[dataCapGba['price_usd_per_m2'] < 10000,:]
dataCapGbamayoresA1000 = dataCapGbaMenoresA10K.loc[dataCapGbaMenoresA10K['price_usd_per_m2'] > 1000,:]
dataCapGbamayoresA1000.sort_values('price_usd_per_m2', ascending = False)

In [ ]:
dataCapGbaSupmayor5 = dataCapGbamayoresA1000.loc[dataCapGbamayoresA1000['surface_total_in_m2'] > 5, :]
dataCapGbaSupmayor5.loc[:, "year"] = dataCapGbaSupmayor5.apply(lambda x: x["created_on"].partition("-")[0], axis = 1)

## Precio por m2 en relacion a la cercania al Subte H y al año de su inauguracion
##### Lo voy a hacer para cada estacion por separado tambien asi se ve mejor

In [ ]:
cercanasANuevasEstaciones = dataCapGbaSupmayor5.loc[dataCapGbaSupmayor5.lat.notnull() & dataCapGbaSupmayor5.lon.notnull(), ['year', 'place_name', 'lat', 'lon', 'price_usd_per_m2']]
cercanasANuevasEstaciones.loc[:, 'distanciaAEstacion'] = cercanasANuevasEstaciones.apply(distanciaANuevasEstaciones, axis = 1)
#Solo dejo las propiedades que estan a menos de 600mts
cercanasANuevasEstaciones = cercanasANuevasEstaciones.loc[cercanasANuevasEstaciones["distanciaAEstacion"] < 600, :]

cercanasGrafico = cercanasANuevasEstaciones.drop('lat', axis=1).drop('lon', axis=1).drop('place_name', axis=1).drop('distanciaAEstacion', axis=1)
cercanasGrafico.groupby('year').agg(np.mean).unstack(level=0).plot(kind ='bar')

## Separando por zonas GBA Norte/Sur/Oeste y Capital Federal

In [ ]:
publicaciones = dataCapGbaSupmayor5.loc[:, ['state_name', 'property_type', 'year', 'price_usd_per_m2']].groupby(['state_name', 'property_type', 'year']).count()
publicacionesCapFed = dataCapGbaSupmayor5.loc[dataCapGbaSupmayor5['state_name'].str.contains('Capital Federal'), ['property_type', 'year', 'price_usd_per_m2']].groupby(['property_type', 'year']).count()
publicacionesGBANorte = dataCapGbaSupmayor5.loc[dataCapGbaSupmayor5['state_name'].str.contains('Bs.As. G.B.A. Zona Norte'), ['property_type', 'year', 'price_usd_per_m2']].groupby(['property_type', 'year']).count()
publicacionesGBASur = dataCapGbaSupmayor5.loc[dataCapGbaSupmayor5['state_name'].str.contains('Bs.As. G.B.A. Zona Sur'), ['property_type', 'year', 'price_usd_per_m2']].groupby(['property_type', 'year']).count()
publicacionesGBAOeste = dataCapGbaSupmayor5.loc[dataCapGbaSupmayor5['state_name'].str.contains('Bs.As. G.B.A. Zona Oeste'), ['property_type', 'year', 'price_usd_per_m2']].groupby(['property_type', 'year']).count()

publicaciones = publicaciones.rename(columns = {'price_usd_per_m2': 'Cantidad de publicaciones'})
plt.figure(figsize = (15,15))
publicaciones.unstack(level=0).plot(kind='bar', subplots=True)

In [ ]:
##Agrupando por barrio de CABA y año y tomando unicamente los que tengan mas de 10 publicaciones por año
CABAPorBarrioYAnio = dataCapGbaSupmayor5.loc[dataCapGbaSupmayor5.state_name.str.contains('Capital Federal'), :]
CABAPorBarrioYAnio.loc[:, "year"] = CABAPorBarrioYAnio.apply(lambda x: x["created_on"].partition("-")[0], axis = 1)
CABAPorBarrioYAnio = CABAPorBarrioYAnio.loc[:, ["year", "place_name", "price_usd_per_m2"]]
CABAPorBarrioYAnio = CABAPorBarrioYAnio.groupby(["place_name", "year"]).agg([np.mean, np.size])
CABAPorBarrioYAnio = CABAPorBarrioYAnio.loc[CABAPorBarrioYAnio['price_usd_per_m2']['size'] > 10, :]
#CABAPorBarrioYAnio.loc[CABAPorBarrioYAnio['price_usd_per_m2']>=CABAPorBarrioYAnio.groupby(['place_name','year']).transform(np.mean)['price_usd_per_m2'],]
CABAPorBarrioYAnio

In [ ]:
CABAPorBarrioYAnio.plot(kind='bar', figsize=(120,18))
plt.bar(CABAPorBarrioYAnio.index.values, CABAPorBarrioYAnio['price_usd_per_m2']['mean'])
plt.suptitle("Precio promedio m2 por barrio y año")
plt.xlabel("Barrio por Año")
plt.ylabel("Precio USD promedio del m2")